In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import os

# Set the column width
pd.set_option('max_colwidth', 400)

### Extract the crowdfunding.xlsx Data

In [2]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel("../Resources/crowdfunding.xlsx")
crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays


In [3]:
# Get a brief summary of the crowdfunding_info DataFrame.
crowdfunding_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cf_id                    1000 non-null   int64 
 1   contact_id               1000 non-null   int64 
 2   company_name             1000 non-null   object
 3   blurb                    1000 non-null   object
 4   goal                     1000 non-null   int64 
 5   pledged                  1000 non-null   int64 
 6   outcome                  1000 non-null   object
 7   backers_count            1000 non-null   int64 
 8   country                  1000 non-null   object
 9   currency                 1000 non-null   object
 10  launched_at              1000 non-null   int64 
 11  deadline                 1000 non-null   int64 
 12  staff_pick               1000 non-null   bool  
 13  spotlight                1000 non-null   bool  
 14  category & sub-category  1000 non-null   

In [4]:
# Convert "goal" and "pledged" to Float
crowdfunding_info_df["goal"] = crowdfunding_info_df["goal"].astype(float)
crowdfunding_info_df["pledged"] = crowdfunding_info_df["pledged"].astype(float)
crowdfunding_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cf_id                    1000 non-null   int64  
 1   contact_id               1000 non-null   int64  
 2   company_name             1000 non-null   object 
 3   blurb                    1000 non-null   object 
 4   goal                     1000 non-null   float64
 5   pledged                  1000 non-null   float64
 6   outcome                  1000 non-null   object 
 7   backers_count            1000 non-null   int64  
 8   country                  1000 non-null   object 
 9   currency                 1000 non-null   object 
 10  launched_at              1000 non-null   int64  
 11  deadline                 1000 non-null   int64  
 12  staff_pick               1000 non-null   bool   
 13  spotlight                1000 non-null   bool   
 14  category & sub-category  

In [5]:
# Convert "launched_at" to "launch_date" and "deadline" to end_date" <-- Convert both to DateTime
crowdfunding_info_df["launched_at"] = pd.to_datetime(crowdfunding_info_df.launched_at*1e9)
crowdfunding_info_df["deadline"] = pd.to_datetime(crowdfunding_info_df.deadline*1e9)
crowdfunding_info_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   cf_id                    1000 non-null   int64         
 1   contact_id               1000 non-null   int64         
 2   company_name             1000 non-null   object        
 3   blurb                    1000 non-null   object        
 4   goal                     1000 non-null   float64       
 5   pledged                  1000 non-null   float64       
 6   outcome                  1000 non-null   object        
 7   backers_count            1000 non-null   int64         
 8   country                  1000 non-null   object        
 9   currency                 1000 non-null   object        
 10  launched_at              1000 non-null   datetime64[ns]
 11  deadline                 1000 non-null   datetime64[ns]
 12  staff_pick               1000 non-n

In [6]:
# Break up category and sub_category
split_columns = crowdfunding_info_df['category & sub-category'].str.split('/', expand=True)
crowdfunding_info_df['category'] = split_columns[0]
crowdfunding_info_df['subcategory'] = split_columns[1] if split_columns.shape[1] > 1 else None

#Drop old column
crowdfunding_info_df.drop(columns=["category & sub-category"], inplace=True)

crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13 06:00:00,2021-03-01 06:00:00,False,False,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25 06:00:00,2021-05-25 05:00:00,False,True,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17 06:00:00,2021-12-30 06:00:00,False,False,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21 05:00:00,2022-01-17 06:00:00,False,False,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21 06:00:00,2021-08-23 05:00:00,False,False,theater,plays


### Create the Category and Subcategory DataFrames
---
**Create a Category DataFrame that has the following columns:**
* A "category_id" column that has entries going sequentially from "cat1" to "cat*n*", where *n* is the number of unique categories
* A "category" column that contains only the category titles

Export the DataFrame as a `category.csv` CSV file.

**Create a SubCategory DataFrame that has the following columns:**
* A "subcategory_id" column that has entries going sequentially from "subcat1" to "subcat*n*", where *n* is the number of unique subcategories
* A "subcategory" column that contains only the subcategory titles

Export the DataFrame as a `subcategory.csv` CSV file.

In [7]:
# Get the crowdfunding_info_df columns.
crowdfunding_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cf_id          1000 non-null   int64         
 1   contact_id     1000 non-null   int64         
 2   company_name   1000 non-null   object        
 3   blurb          1000 non-null   object        
 4   goal           1000 non-null   float64       
 5   pledged        1000 non-null   float64       
 6   outcome        1000 non-null   object        
 7   backers_count  1000 non-null   int64         
 8   country        1000 non-null   object        
 9   currency       1000 non-null   object        
 10  launched_at    1000 non-null   datetime64[ns]
 11  deadline       1000 non-null   datetime64[ns]
 12  staff_pick     1000 non-null   bool          
 13  spotlight      1000 non-null   bool          
 14  category       1000 non-null   object        
 15  subcategory    1000 no

In [8]:
# Assign the category and subcategory values to category and subcategory columns.
categories = crowdfunding_info_df["category"].copy()
subcategories = crowdfunding_info_df["subcategory"].copy()

In [9]:
# Get the unique categories and subcategories in separate lists.
categories.unique()
categories_lists = pd.Series(categories).unique()
categories_lists

array(['food', 'music', 'technology', 'theater', 'film & video',
       'publishing', 'games', 'photography', 'journalism'], dtype=object)

In [10]:
subcategories.unique()
subcategories_lists = pd.Series(subcategories).unique()
subcategories_lists

array(['food trucks', 'rock', 'web', 'plays', 'documentary',
       'electric music', 'drama', 'indie rock', 'wearables', 'nonfiction',
       'animation', 'video games', 'shorts', 'fiction',
       'photography books', 'radio & podcasts', 'metal', 'jazz',
       'translations', 'television', 'mobile games', 'world music',
       'science fiction', 'audio'], dtype=object)

In [11]:
# Get the number of distinct values in the categories and subcategories lists.
category_count = len(categories_lists)
category_count

9

In [12]:
subcategory_count = len(subcategories_lists)
subcategory_count

24

In [13]:
# Create numpy arrays from 1-9 for the categories and 1-24 for the subcategories.
category_ids = np.arange(1, 10)
subcategory_ids = np.arange(1, 25)

# Initialize empty dictionaries to store the unique mappings
category_mapping = {}
subcategory_mapping = {}

# For loop to assign categories to category IDs
for i in range(len(category_ids)):
    if i < len(categories_lists):
        category_mapping[category_ids[i]] = categories_lists[i]

    else:
        print(f"Duplicate category ID found: {category_ids[i]}")


category_mapping

{1: 'food',
 2: 'music',
 3: 'technology',
 4: 'theater',
 5: 'film & video',
 6: 'publishing',
 7: 'games',
 8: 'photography',
 9: 'journalism'}

In [14]:
# For loop to assign subcategories to subcategory IDs
for i in range(len(subcategory_ids)):
        if i < len(subcategories_lists):
            subcategory_mapping[subcategory_ids[i]] = subcategories_lists[i]

        else:
            print(f"Duplicate category ID found: {subcategory_ids[i]}")

subcategory_mapping

{1: 'food trucks',
 2: 'rock',
 3: 'web',
 4: 'plays',
 5: 'documentary',
 6: 'electric music',
 7: 'drama',
 8: 'indie rock',
 9: 'wearables',
 10: 'nonfiction',
 11: 'animation',
 12: 'video games',
 13: 'shorts',
 14: 'fiction',
 15: 'photography books',
 16: 'radio & podcasts',
 17: 'metal',
 18: 'jazz',
 19: 'translations',
 20: 'television',
 21: 'mobile games',
 22: 'world music',
 23: 'science fiction',
 24: 'audio'}

In [15]:
# Use a list comprehension to add "cat" to each category_id.
cat_ids = [f"cat{category_id}" for category_id in range(1, len(categories_lists) + 1)]


# Initialize the mapping dictionary
category_mapping = {}
[category_mapping.update({cat_ids[i]: categories_lists[i]}) for i in range(len(categories_lists))]
print(category_mapping)

{'cat1': 'food', 'cat2': 'music', 'cat3': 'technology', 'cat4': 'theater', 'cat5': 'film & video', 'cat6': 'publishing', 'cat7': 'games', 'cat8': 'photography', 'cat9': 'journalism'}


In [16]:
# Use a list comprehension to add "cat" to each subcategory_id.
scat_ids = [f"cat{subcategory_id}" for subcategory_id in range(1, len(subcategories_lists) + 1)]

subcategory_mapping = {}
[subcategory_mapping.update({scat_ids[i]: subcategories_lists[i]}) for i in range(len(subcategories_lists))]
print(subcategory_mapping)

{'cat1': 'food trucks', 'cat2': 'rock', 'cat3': 'web', 'cat4': 'plays', 'cat5': 'documentary', 'cat6': 'electric music', 'cat7': 'drama', 'cat8': 'indie rock', 'cat9': 'wearables', 'cat10': 'nonfiction', 'cat11': 'animation', 'cat12': 'video games', 'cat13': 'shorts', 'cat14': 'fiction', 'cat15': 'photography books', 'cat16': 'radio & podcasts', 'cat17': 'metal', 'cat18': 'jazz', 'cat19': 'translations', 'cat20': 'television', 'cat21': 'mobile games', 'cat22': 'world music', 'cat23': 'science fiction', 'cat24': 'audio'}


In [17]:
# Create category_mapping with a limit on length
category_mapping = {}
[category_mapping.update({cat_ids[i]: categories_lists[i]}) for i in range(len(categories_lists))]

# Use only the truncated categories for the DataFrame
truncated_categories = list(category_mapping.values())

# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
category_df = pd.DataFrame({
    'category_id': range(1, len(categories_lists) + 1),
    'category_name': categories_lists
})

# Create a category DataFrame with the subcategory_id array as the subcategory_id and subcategories list as the subcategory name.
subcategory_mapping = {}
[subcategory_mapping.update({scat_ids[i]: subcategories_lists[i]}) for i in range(len(subcategories_lists))]

# Use only the truncated categories for the DataFrame
truncated_subcategories = list(subcategory_mapping.values())

# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
subcategory_df = pd.DataFrame({
    'subcategory_id': range(1, len(subcategories_lists) + 1),
    'subcategory_name': subcategories_lists
})


In [18]:
# View the category_df
category_df

,category_id,category_name
0,1,food
1,2,music
2,3,technology
3,4,theater
4,5,film & video
5,6,publishing
6,7,games
7,8,photography
8,9,journalism


In [19]:
# View the subcategory_df
subcategory_df

,subcategory_id,subcategory_name
0,1,food trucks
1,2,rock
2,3,web
3,4,plays
4,5,documentary
5,6,electric music
6,7,drama
7,8,indie rock
8,9,wearables
9,10,nonfiction


In [20]:
# Absolute path to the existing 'Resources' folder
directory = r"C:/Users/Chase/bootcamp/hw/Projects/project2/Chase/project-ETL-crowdfunding/Submission/Resources"

# Save the DataFrames to CSV files
category_df.to_csv(os.path.join(directory, "category.csv"), index=False)
subcategory_df.to_csv(os.path.join(directory, "subcategory.csv"), index=False)

### Campaign DataFrame
----
**Create a Campaign DataFrame that has the following columns:**
* The "cf_id" column
* The "contact_id" column
* The "company_name" column
* The "blurb" column, renamed to "description"
* The "goal" column, converted to the `float` data type
* The "pledged" column, converted to the `float` data type
* The "outcome" column
* The "backers_count" column
* The "country" column
* The "currency" column
* The "launched_at" column, renamed to "launch_date" and with the UTC times converted to the `datetime` format
* The "deadline" column, renamed to "end_date" and with the UTC times converted to the `datetime` format
* The "category_id" column, with unique identification numbers matching those in the "category_id" column of the category DataFrame
* The "subcategory_id" column, with the unique identification numbers matching those in the "subcategory_id" column of the subcategory DataFrame
 

Then export the DataFrame as a `campaign.csv` CSV file.


In [21]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df.
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13 06:00:00,2021-03-01 06:00:00,False,False,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25 06:00:00,2021-05-25 05:00:00,False,True,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17 06:00:00,2021-12-30 06:00:00,False,False,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21 05:00:00,2022-01-17 06:00:00,False,False,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21 06:00:00,2021-08-23 05:00:00,False,False,theater,plays


In [22]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df["quote"] = campaign_df["blurb"]
campaign_df["launch_date"] = campaign_df["launched_at"]
campaign_df["end_date"] = campaign_df["deadline"]

# Drop the original columns
campaign_df.drop(columns=["blurb", "launched_at", "deadline"], inplace=True)

campaign_df.head()

,cf_id,contact_id,company_name,goal,pledged,outcome,backers_count,country,currency,staff_pick,spotlight,category,subcategory,quote,launch_date,end_date
0,147,4661,"Baldwin, Riley and Jackson",100.0,0.0,failed,0,CA,CAD,False,False,food,food trucks,Pre-emptive tertiary standardization,2020-02-13 06:00:00,2021-03-01 06:00:00
1,1621,3765,Odom Inc,1400.0,14560.0,successful,158,US,USD,False,True,music,rock,Managed bottom-line architecture,2021-01-25 06:00:00,2021-05-25 05:00:00
2,1812,4187,"Melton, Robinson and Fritz",108400.0,142523.0,successful,1425,AU,AUD,False,False,technology,web,Function-based leadingedge pricing structure,2020-12-17 06:00:00,2021-12-30 06:00:00
3,2156,4941,"Mcdonald, Gonzalez and Ross",4200.0,2477.0,failed,24,US,USD,False,False,music,rock,Vision-oriented fresh-thinking conglomeration,2021-10-21 05:00:00,2022-01-17 06:00:00
4,1365,2199,Larson-Little,7600.0,5265.0,failed,53,US,USD,False,False,theater,plays,Proactive foreground core,2020-12-21 06:00:00,2021-08-23 05:00:00


In [23]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df['goal'] = campaign_df['goal'].astype(float)
campaign_df['pledged'] = campaign_df['pledged'].astype(float)

In [24]:
# Check the datatypes
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cf_id          1000 non-null   int64         
 1   contact_id     1000 non-null   int64         
 2   company_name   1000 non-null   object        
 3   goal           1000 non-null   float64       
 4   pledged        1000 non-null   float64       
 5   outcome        1000 non-null   object        
 6   backers_count  1000 non-null   int64         
 7   country        1000 non-null   object        
 8   currency       1000 non-null   object        
 9   staff_pick     1000 non-null   bool          
 10  spotlight      1000 non-null   bool          
 11  category       1000 non-null   object        
 12  subcategory    1000 non-null   object        
 13  quote          1000 non-null   object        
 14  launch_date    1000 non-null   datetime64[ns]
 15  end_date       1000 no

In [25]:
# Format the launch_date and end_date columns to datetime format
campaign_df['launch_date'] = pd.to_datetime(campaign_df['launch_date'])
campaign_df['end_date'] = pd.to_datetime(campaign_df['end_date'])
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   cf_id          1000 non-null   int64         
 1   contact_id     1000 non-null   int64         
 2   company_name   1000 non-null   object        
 3   goal           1000 non-null   float64       
 4   pledged        1000 non-null   float64       
 5   outcome        1000 non-null   object        
 6   backers_count  1000 non-null   int64         
 7   country        1000 non-null   object        
 8   currency       1000 non-null   object        
 9   staff_pick     1000 non-null   bool          
 10  spotlight      1000 non-null   bool          
 11  category       1000 non-null   object        
 12  subcategory    1000 non-null   object        
 13  quote          1000 non-null   object        
 14  launch_date    1000 non-null   datetime64[ns]
 15  end_date       1000 no

In [26]:
# Merge the campaign_df with the category_df on the "category" column and the subcategory_df on the "subcategory" column.
merged_df = pd.merge(campaign_df, category_df, left_on='category', right_on='category_name', how='left')
merged_df = pd.merge(merged_df, subcategory_df, left_on='subcategory', right_on='subcategory_name', how='left')

# Display the last 10 rows of the merged DataFrame
merged_df.tail(10)

,cf_id,contact_id,company_name,goal,pledged,outcome,backers_count,country,currency,staff_pick,spotlight,category,subcategory,quote,launch_date,end_date,category_id,category_name,subcategory_id,subcategory_name
990,1561,5500,Ortiz-Roberts,7800.0,6839.0,failed,64,US,USD,False,True,film & video,drama,Devolved foreground customer loyalty,2021-06-09 05:00:00,2021-06-18 05:00:00,5,film & video,7,drama
991,2632,3494,Ramirez LLC,9800.0,11091.0,successful,241,US,USD,False,True,music,rock,Reduced reciprocal focus group,2020-12-09 06:00:00,2021-05-26 05:00:00,2,music,2,rock
992,439,3924,Morrow Inc,3100.0,13223.0,successful,132,US,USD,False,True,film & video,drama,Networked global migration,2020-06-14 05:00:00,2021-02-09 06:00:00,5,film & video,7,drama
993,461,3521,Erickson-Rogers,9800.0,7608.0,canceled,75,IT,EUR,False,True,photography,photography books,De-engineered even-keeled definition,2021-07-03 05:00:00,2021-07-08 05:00:00,8,photography,15,photography books
994,1419,5262,"Leach, Rich and Price",141100.0,74073.0,failed,842,US,USD,False,True,publishing,translations,Implemented bi-directional flexibility,2021-11-15 06:00:00,2021-12-07 06:00:00,6,publishing,19,translations
995,2986,3684,Manning-Hamilton,97300.0,153216.0,successful,2043,US,USD,False,True,food,food trucks,Vision-oriented scalable definition,2020-12-29 06:00:00,2021-05-30 05:00:00,1,food,1,food trucks
996,2031,5784,Butler LLC,6600.0,4814.0,failed,112,US,USD,False,False,theater,plays,Future-proofed upward-trending migration,2021-10-15 05:00:00,2021-11-30 06:00:00,4,theater,4,plays
997,1627,1498,Ball LLC,7600.0,4603.0,canceled,139,IT,EUR,False,False,theater,plays,Right-sized full-range throughput,2021-11-06 05:00:00,2021-12-10 06:00:00,4,theater,4,plays
998,2175,6073,"Taylor, Santiago and Flores",66600.0,37823.0,failed,374,US,USD,False,True,music,indie rock,Polarized composite customer loyalty,2020-10-08 05:00:00,2021-04-11 05:00:00,2,music,8,indie rock
999,1788,4939,"Hernandez, Norton and Kelley",111100.0,62819.0,canceled,1122,US,USD,False,False,food,food trucks,Expanded eco-centric policy,2020-12-30 06:00:00,2021-08-18 05:00:00,1,food,1,food trucks


In [27]:
# Drop unwanted columns, including 'staff_pick' and 'spotlight'
merged_df.drop(columns=["staff_pick", "spotlight"], inplace=True)
merged_df.head()

,cf_id,contact_id,company_name,goal,pledged,outcome,backers_count,country,currency,category,subcategory,quote,launch_date,end_date,category_id,category_name,subcategory_id,subcategory_name
0,147,4661,"Baldwin, Riley and Jackson",100.0,0.0,failed,0,CA,CAD,food,food trucks,Pre-emptive tertiary standardization,2020-02-13 06:00:00,2021-03-01 06:00:00,1,food,1,food trucks
1,1621,3765,Odom Inc,1400.0,14560.0,successful,158,US,USD,music,rock,Managed bottom-line architecture,2021-01-25 06:00:00,2021-05-25 05:00:00,2,music,2,rock
2,1812,4187,"Melton, Robinson and Fritz",108400.0,142523.0,successful,1425,AU,AUD,technology,web,Function-based leadingedge pricing structure,2020-12-17 06:00:00,2021-12-30 06:00:00,3,technology,3,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",4200.0,2477.0,failed,24,US,USD,music,rock,Vision-oriented fresh-thinking conglomeration,2021-10-21 05:00:00,2022-01-17 06:00:00,2,music,2,rock
4,1365,2199,Larson-Little,7600.0,5265.0,failed,53,US,USD,theater,plays,Proactive foreground core,2020-12-21 06:00:00,2021-08-23 05:00:00,4,theater,4,plays


In [28]:
# Export the DataFrame as a CSV file.
# Absolute path to the existing 'Resources' folder
directory = r"C:/Users/Chase/bootcamp/hw/Projects/project2/Chase/project-ETL-crowdfunding/Submission/Resources"

# Save the DataFrames to CSV files
merged_df.to_csv(os.path.join(directory, "cleaned_campaign.csv"), index=False)

### Extract the contacts.xlsx Data.

In [29]:
# Read the data into a Pandas DataFrame. Use the `header=3` parameter when reading in the data.
contact_info_df = pd.read_excel(r"C:/Users/Chase/bootcamp/hw/Projects/project2/Chase/project-ETL-crowdfunding/Submission/Resources/contacts.xlsx", header=3)
contact_info_df.head()

,contact_info
0,"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}"
1,"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}"
2,"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}"
3,"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}"
4,"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}"


### Create the Contacts DataFrame 
---
**Create a Contacts DataFrame that has the following columns:**
- A column named "contact_id"  that contains the unique number of the contact person.
- A column named "first_name" that contains the first name of the contact person.
- A column named "last_name" that contains the first name of the contact person.
- A column named "email" that contains the email address of the contact person

Then export the DataFrame as a `contacts.csv` CSV file.

### Option 1: Use Pandas to create the contacts DataFrame.

In [31]:
import json
dict_values = []

# Iterate over each row in the DataFrame
for _, row in contact_info_df.iterrows():
    # Parse the string as JSON
    contact_dict = json.loads(row[0])
    
    # Extract the contact details and split the name into first and last names
    contact_id = contact_dict.get("contact_id")
    name = contact_dict.get("name")
    email = contact_dict.get("email")
    
    # Append the raw data to the list
    dict_values.append({
        "contact_id": contact_id,
        "name": name,
        "email": email
    })

C:\Users\Chase\AppData\Local\Temp\ipykernel_852\1869202774.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contact_dict = json.loads(row[0])


In [32]:
# Create a contacts_df DataFrame from the list of dictionaries
contacts_df = pd.DataFrame(dict_values)

In [33]:
# Check the datatypes.
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   name        1000 non-null   object
 2   email       1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [36]:
# Create a "first"name" and "last_name" column with the first and last names from the "name" column.
contacts_df['name'] = contacts_df['name'].astype(str)

# Split the 'name' column into 'first_name' and 'last_name'
contacts_df[['first_name', 'last_name']] = contacts_df['name'].str.split(' ', n=1, expand=True)

#Drop the name column
contacts_df.drop(columns=["name"], inplace=True)

contacts_df.head()

,contact_id,email,first_name,last_name
0,4661,cecilia.velasco@rodrigues.fr,Cecilia,Velasco
1,3765,mariana.ellis@rossi.org,Mariana,Ellis
2,4187,sofie.woods@riviere.com,Sofie,Woods
3,4941,jeanette.iannotti@yahoo.com,Jeanette,Iannotti
4,2199,samuel.sorgatz@gmail.com,Samuel,Sorgatz


In [37]:
# Reorder the columns
column_order = ['contact_id', 'first_name', 'last_name', 'email']
contacts_df = contacts_df[column_order]
contacts_df.head()

,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com


In [38]:
# Check the datatypes one more time before exporting as CSV file.
contacts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [40]:
# Export the DataFrame as a CSV file.
# Absolute path to the existing 'Resources' folder
directory = r"C:/Users/Chase/bootcamp/hw/Projects/project2/Chase/project-ETL-crowdfunding/Submission/Resources"

# Save the DataFrames to CSV files
merged_df.to_csv(os.path.join(directory, "contacts_df_clean.csv"), index=False)

### Option 2: Use regex to create the contacts DataFrame.

In [ ]:
contact_info_df_copy = contact_info_df.copy()
contact_info_df_copy.head()

In [ ]:
# Extract the four-digit contact ID number.


In [ ]:
# Check the datatypes.


In [ ]:
# Convert the "contact_id" column to an int64 data type using `pd.to_numeric`.


In [ ]:
# Extract the name of the contact and add it to a new column.


In [ ]:
# Extract the email from the contacts and add the values to a new column.


In [ ]:
# Create a copy of the contact_info_df with the 'contact_id', 'name', 'email' columns.


In [ ]:
# Create a "first"name" and "last_name" column with the first and last names from the "name" column.


# Drop the contact_name column


In [ ]:
# Reorder the columns


In [ ]:
# Check the datatypes one more time before exporting as CSV file.


In [ ]:
# Export the DataFrame as a CSV file.
# contacts_df_clean.to_csv("Resources/contacts.csv", encoding='utf8', index=False)